In [423]:
import pandas as pd

In [424]:
import numpy as np

In [449]:
import matplotlib.pyplot as plt 

In [450]:
import os

In [451]:
os.listdir('Train')

['data_description.txt', 'Train_Data.csv']

In [452]:
from sklearn.preprocessing import OneHotEncoder
oht = OneHotEncoder()

In [453]:
X = pd.read_csv('Train/Train_Data.csv')
test_df = pd.read_csv('Test(1)/Test_Data.csv')
X = pd.concat([X,test_df], sort = False)

In [454]:
drop_col = ['Id' ,'Alley' , 'PoolQC' , 'Fence' , 'MiscFeature']

In [455]:
X = X.drop(columns=drop_col)

In [456]:
X['LotFrontage'] = X['LotFrontage'].fillna(X['LotFrontage'].mean())
X['GarageYrBlt'] = X['GarageYrBlt'].fillna(X['GarageYrBlt'].mean())
X['MasVnrArea'] = X['MasVnrArea'].fillna(X['MasVnrArea'].mean())
c = 'GarageCars'
X[c] = X[c].fillna(X[c].mean())

In [457]:
X['MasVnrType'] = X['MasVnrType'].fillna(X['MasVnrType'].mode()[0])
X['BsmtQual'] = X['BsmtQual'].fillna(X['BsmtQual'].mode()[0])
X['BsmtCond'] = X['BsmtCond'].fillna(X['BsmtCond'].mode()[0])
X['BsmtExposure'] = X['BsmtExposure'].fillna(X['BsmtExposure'].mode()[0])
X['BsmtFinType1'] = X['BsmtFinType1'].fillna(X['BsmtFinType1'].mode()[0])
X['FireplaceQu'] = X['FireplaceQu'].fillna(X['FireplaceQu'].mode()[0])
X['GarageType'] = X['GarageType'].fillna(X['GarageType'].mode()[0])
X['BsmtFinType2'] = X['BsmtFinType2'].fillna(X['BsmtFinType2'].mode()[0])
X['GarageQual'] = X['GarageQual'].fillna(X['GarageQual'].mode()[0])
X['GarageCond'] = X['GarageCond'].fillna(X['GarageCond'].mode()[0])
X['GarageFinish'] = X['GarageFinish'].fillna(X['GarageFinish'].mode()[0])
X['Electrical'] = X['Electrical'].fillna(X['Electrical'].mode()[0])
c1 = 'Functional'
X[c1] = X[c1].fillna(X[c1].mode()[0])

In [458]:
X = pd.DataFrame(X.values)

In [459]:
dummydf = pd.DataFrame()
frames = []
for col in X.columns:
    if type(X[col].iloc[0]) != str:
        frames.append(pd.DataFrame(X[col]))
    else:
        oht_df = pd.DataFrame(oht.fit_transform(X[col].values.reshape(-1,1)).toarray())
        frames.append(oht_df)
dummydf = pd.concat(frames,axis=1, sort=False)

In [460]:
dummydf.values

array([[60, 0.0, 0.0, ..., 1.0, 0.0, 208500.0],
       [20, 0.0, 0.0, ..., 1.0, 0.0, 181500.0],
       [60, 0.0, 0.0, ..., 1.0, 0.0, 223500.0],
       ...,
       [70, 0.0, 0.0, ..., 1.0, 0.0, nan],
       [20, 0.0, 0.0, ..., 1.0, 0.0, nan],
       [20, 0.0, 0.0, ..., 1.0, 0.0, nan]], dtype=object)

In [461]:
dummydf = dummydf.loc[:,~dummydf.columns.duplicated()]
X_=dummydf.values[:1100,:-1]
y_=dummydf.values[:1100,-1]

In [462]:
X_ = (X_-X_.mean())/X_.std()
y_ = (y_-y_.mean())/y_.std() 

# Model

In [463]:
import keras

In [464]:
from keras.models import Sequential

In [465]:
from keras.layers import *

In [466]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)

In [467]:
from keras.callbacks import ModelCheckpoint

In [468]:
model = Sequential()
leakyrelu = LeakyReLU(0.3)
model.add(Dense(100, input_shape = (53,)))
model.add(Dense(512 ))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(16))
model.add(Dense(1))


model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_145 (Dense)            (None, 100)               5400      
_________________________________________________________________
dense_146 (Dense)            (None, 512)               51712     
_________________________________________________________________
dense_147 (Dense)            (None, 128)               65664     
_________________________________________________________________
dense_148 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_149 (Dense)            (None, 16)                1040      
_________________________________________________________________
dense_150 (Dense)            (None, 1)                 17        
Total params: 132,089
Trainable params: 132,089
Non-trainable params: 0
_______________________________________________

In [469]:
model.compile(optimizer = 'adam' , loss = 'mean_squared_error' , metrics=['mse','mae'])

In [470]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_mae', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

hist = model.fit(X_train, y_train , batch_size=32 ,epochs=20 ,validation_data=(X_test,y_test) , callbacks=callbacks_list)

Train on 880 samples, validate on 220 samples
Epoch 1/20
880/880 [==============================] - 0s 247us/step - loss: 4.8674 - mse: 4.8674 - mae: 1.5915 - val_loss: 0.4715 - val_mse: 0.4715 - val_mae: 0.4541

Epoch 00001: val_mae improved from inf to 0.45409, saving model to best_model.h5
Epoch 2/20
880/880 [==============================] - 0s 111us/step - loss: 0.3355 - mse: 0.3355 - mae: 0.4342 - val_loss: 0.5218 - val_mse: 0.5218 - val_mae: 0.4567

Epoch 00002: val_mae did not improve from 0.45409
Epoch 3/20
880/880 [==============================] - 0s 150us/step - loss: 0.2584 - mse: 0.2584 - mae: 0.3599 - val_loss: 0.5212 - val_mse: 0.5212 - val_mae: 0.4632

Epoch 00003: val_mae did not improve from 0.45409
Epoch 4/20
880/880 [==============================] - 0s 117us/step - loss: 0.2402 - mse: 0.2402 - mae: 0.3446 - val_loss: 0.4680 - val_mse: 0.4680 - val_mae: 0.4308

Epoch 00004: val_mae improved from 0.45409 to 0.43078, saving model to best_model.h5
Epoch 5/20
880/880 [

In [471]:
model.load_weights('best_model.h5')

In [472]:
model.evaluate(X_test,y_test)

220/220 [==============================] - 0s 45us/step


[0.40999896878545933, 0.40999895334243774, 0.4136522114276886]

In [448]:
"""test_df['LotFrontage'] = test_df['LotFrontage'].fillna(test_df['LotFrontage'].mean())
test_df['GarageYrBlt'] = test_df['GarageYrBlt'].fillna(test_df['GarageYrBlt'].mean())
test_df['MasVnrArea'] = test_df['MasVnrArea'].fillna(test_df['MasVnrArea'].mean())
test_df['MasVnrType'] = test_df['MasVnrType'].fillna(test_df['MasVnrType'].mode()[0])
test_df['BsmtQual'] = test_df['BsmtQual'].fillna(test_df['BsmtQual'].mode()[0])
test_df['BsmtCond'] = test_df['BsmtCond'].fillna(test_df['BsmtCond'].mode()[0])
test_df['BsmtExposure'] = test_df['BsmtExposure'].fillna(test_df['BsmtExposure'].mode()[0])
test_df['BsmtFinType1'] = test_df['BsmtFinType1'].fillna(test_df['BsmtFinType1'].mode()[0])
test_df['FireplaceQu'] = test_df['FireplaceQu'].fillna(test_df['FireplaceQu'].mode()[0])
test_df['GarageType'] = test_df['GarageType'].fillna(test_df['GarageType'].mode()[0])
test_df['BsmtFinType2'] = test_df['BsmtFinType2'].fillna(test_df['BsmtFinType2'].mode()[0])
test_df['GarageQual'] = test_df['GarageQual'].fillna(test_df['GarageQual'].mode()[0])
test_df['GarageCond'] = test_df['GarageCond'].fillna(test_df['GarageCond'].mode()[0])
test_df['GarageFinish'] = test_df['GarageFinish'].fillna(test_df['GarageFinish'].mode()[0])
test_df['Electrical'] = test_df['Electrical'].fillna(test_df['Electrical'].mode()[0])"""

"test_df['LotFrontage'] = test_df['LotFrontage'].fillna(test_df['LotFrontage'].mean())\ntest_df['GarageYrBlt'] = test_df['GarageYrBlt'].fillna(test_df['GarageYrBlt'].mean())\ntest_df['MasVnrArea'] = test_df['MasVnrArea'].fillna(test_df['MasVnrArea'].mean())\ntest_df['MasVnrType'] = test_df['MasVnrType'].fillna(test_df['MasVnrType'].mode()[0])\ntest_df['BsmtQual'] = test_df['BsmtQual'].fillna(test_df['BsmtQual'].mode()[0])\ntest_df['BsmtCond'] = test_df['BsmtCond'].fillna(test_df['BsmtCond'].mode()[0])\ntest_df['BsmtExposure'] = test_df['BsmtExposure'].fillna(test_df['BsmtExposure'].mode()[0])\ntest_df['BsmtFinType1'] = test_df['BsmtFinType1'].fillna(test_df['BsmtFinType1'].mode()[0])\ntest_df['FireplaceQu'] = test_df['FireplaceQu'].fillna(test_df['FireplaceQu'].mode()[0])\ntest_df['GarageType'] = test_df['GarageType'].fillna(test_df['GarageType'].mode()[0])\ntest_df['BsmtFinType2'] = test_df['BsmtFinType2'].fillna(test_df['BsmtFinType2'].mode()[0])\ntest_df['GarageQual'] = test_df['Gar

In [479]:
test = dummydf.values[1100:,:-1]

In [480]:
test

array([[30, 0.0, 0.0, ..., 0, 1, 2009],
       [20, 0.0, 0.0, ..., 0, 7, 2007],
       [20, 0.0, 0.0, ..., 0, 4, 2007],
       ...,
       [70, 0.0, 0.0, ..., 2500, 5, 2010],
       [20, 0.0, 0.0, ..., 0, 4, 2010],
       [20, 0.0, 0.0, ..., 0, 6, 2008]], dtype=object)

In [ ]:
dic = {'Id' : np.arange(1101,1461) , 'SalePrice' : np.round(pred.reshape(1,-1)[0])}

In [ ]:
df = pd.DataFrame(dic)

In [ ]:
df.to_csv('Sample_house_prediction.csv', index=False) 

In [ ]:
df = pd.concat([df , X_train['MSSubClass']])